In [4]:
import cv2
import mediapipe as mp
from cvzone.HandTrackingModule import HandDetector
import socket
import json

cap = cv2.VideoCapture(0) # uncomment if you have webcam
# cap = cv2.VideoCapture("http://10.0.0.115:4747/video") # ONLY OPENS UP WHEN ONE INSTANCE OF THIS IS OPEN! DO NOT OPEN IN BROWSER!
IMG_HEIGHT = 720
IMG_WIDTH = 1280 
cap.set(3, IMG_WIDTH)
cap.set(4, IMG_HEIGHT)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
# DRAW BOTH HANDS ON SCREEN
detector = HandDetector(maxHands=2, detectionCon=0.8)

# Comms
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
serverAddressPort = ("127.0.0.1", 5052)

while(True):
    success, img = cap.read()
    # flipping image to make everything easier to control (like a mirror)

    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    
    """
    Structure: {
        "landmarks": List,
        "bbox": Tuple,
        "center": Tuple,
        "type": String
    }
    """
    data = {
        "left": {
            "landmarks": [],
            "bbox": (),
            "center": (),
            "type": ""
        },
        "right": {
            "landmarks": [],
            "bbox": (),
            "center": (),
            "type": ""
        }
    }
    if hands:
        for hand in hands:
            # get left or right from hand so we can access the correct hand in the data object with a single if statement
            lr = hand["type"].lower()

            if (data[lr]["type"] == ""):
                # get list of landmarks, bounding box, center, then add them to JSON object
                lmList = hand["lmList"]
                bbox = hand["bbox"]
                center = hand["center"]

                data[lr]["bbox"] = bbox
                data[lr]["center"] = center
                data[lr]["type"] = lr

                for lm in lmList:
                    data[lr]["landmarks"].extend([lm[0], IMG_HEIGHT - lm[1], lm[2]])

        # convert to JSON
        json_data = json.dumps(data)
        print(json_data)
        sock.sendto(json_data.encode(), serverAddressPort)
    # if hands:
    #     hand = hands[0]
    #     # get list of landmarks
    #     lmList = hand['lmList']

    #     for lm in lmList:
    #         data.extend([lm[0], IMG_HEIGHT - lm[1], lm[2]])
    #     sock.sendto(str.encode(str(data)), serverAddressPort)
    
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

{"left": {"landmarks": [], "bbox": [], "center": [], "type": ""}, "right": {"landmarks": [397, 110, 0, 473, 139, -22, 520, 196, -33, 538, 264, -39, 555, 314, -42, 465, 277, -30, 478, 323, -35, 495, 293, -34, 498, 259, -31, 418, 282, -22, 437, 326, -20, 455, 299, -19, 458, 268, -21, 377, 273, -15, 399, 309, -12, 416, 286, -7, 419, 258, -7, 343, 257, -9, 362, 285, -6, 376, 268, 2, 378, 245, 8], "bbox": [343, 394, 212, 216], "center": [449, 502], "type": "right"}}
{"left": {"landmarks": [], "bbox": [], "center": [], "type": ""}, "right": {"landmarks": [394, 248, 0, 454, 271, -9, 501, 306, -21, 548, 324, -31, 584, 333, -44, 475, 386, -36, 500, 461, -51, 517, 503, -60, 532, 533, -65, 429, 398, -40, 436, 480, -53, 442, 530, -61, 449, 563, -67, 385, 392, -42, 385, 473, -54, 385, 523, -65, 389, 559, -72, 350, 373, -44, 335, 429, -56, 327, 459, -61, 322, 481, -63], "bbox": [322, 157, 262, 315], "center": [453, 314], "type": "right"}}
{"left": {"landmarks": [1130, 325, 0, 1055, 354, -42, 1000, 3

In [7]:
print(hand['lmList'])
print(hand)

[[479, 341, 0], [551, 317, -23], [612, 267, -32], [630, 206, -40], [622, 157, -49], [593, 185, -18], [622, 118, -41], [640, 79, -60], [659, 51, -72], [554, 165, -22], [581, 82, -42], [602, 34, -59], [622, 0, -71], [514, 161, -30], [540, 82, -51], [558, 35, -66], [578, 1, -75], [470, 172, -39], [476, 109, -61], [486, 68, -70], [499, 33, -76]]
{'lmList': [[479, 341, 0], [551, 317, -23], [612, 267, -32], [630, 206, -40], [622, 157, -49], [593, 185, -18], [622, 118, -41], [640, 79, -60], [659, 51, -72], [554, 165, -22], [581, 82, -42], [602, 34, -59], [622, 0, -71], [514, 161, -30], [540, 82, -51], [558, 35, -66], [578, 1, -75], [470, 172, -39], [476, 109, -61], [486, 68, -70], [499, 33, -76]], 'bbox': (470, 0, 189, 341), 'center': (564, 170), 'type': 'Left'}
